### <center> Линейная регрессия, Lasso-регрессия и случайный лес в задаче по определению качества вина</center>



In [1]:
# отключим всякие предупреждения Anaconda
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score,
                                     train_test_split)
from sklearn.preprocessing import StandardScaler

**Будем работать с набором данных по качеству белого вина**

**Загружаем данные**

In [2]:
data = pd.read_csv('winequality-white.csv', sep=';')

In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


**Отделите целевой признак, разделите обучающую выборку в отношении 7:3 (30% - под оставленную выборку, пусть random_state=17) и отмасштабируйте данные с помощью StandardScaler**

In [3]:
y = data['quality']  # Ваш код здесь

X_train, X_holdout, y_train, y_holdout = train_test_split(
    data.drop('quality', axis=1), y, test_size=.3, random_state=17)  # Ваш код здесь
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Ваш код здесь
X_holdout_scaled = scaler.transform(X_holdout)  # Ваш код здесь


## Линейная регрессия

**Обучите простую линейную регрессию**

In [37]:
linreg =LinearRegression() # Ваш код здесь
# linreg.fit(X_train_scaled,y_train) # Ваш код здесь
linreg.fit(X_train,y_train) # Ваш код здесь

LinearRegression()

**<font color='red'>Вопрос 1:</font> Каковы среднеквадратичные ошибки линейной регрессии на обучающей и отложенной выборках?**

In [38]:
# Ваш код здесь
print("Mean squared error (train): %.3f" %
      ((mean_squared_error(y_train, linreg.predict(X_train_scaled)))))
print("Mean squared error (test): %.3f" % ((mean_squared_error(
    y_holdout, linreg.predict(X_holdout_scaled)))))  # Ваш код здесь


Mean squared error (train): 100272.777
Mean squared error (test): 102163.614


**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина (учтите, что большие по модулю отрицательные значения коэффициентов тоже говорят о сильном влиянии). Создайте для этого новый небольшой DataFrame.**<br>
**<font color='red'>Вопрос 2:</font> Какой признак линейная регрессия считает наиболее сильно влияющим на качество вина?**

In [33]:
# linreg_coef = pd.DataFrame # Ваш код здесь
# linreg_coef.sort_values # Ваш код здесь
pd.DataFrame(linreg.coef_, X_holdout.columns, columns=[
             "coef"]).sort_values(by="coef", ascending=False)


,coef
residual sugar,0.538164
pH,0.150036
alcohol,0.129533
fixed acidity,0.097822
sulphates,0.062053
free sulfur dioxide,0.042180
total sulfur dioxide,0.014304
chlorides,0.008127
citric acid,-0.000183
volatile acidity,-0.192260


## Lasso-регрессия

**Обучите Lasso-регрессию с небольшим коэффициентом $\alpha = 0.01$ (слабая регуляризация). Пусть опять random_state=17.**

In [9]:
lasso1 = Lasso(alpha=.01,random_state=17) # Ваш код здесь
lasso1.fit(X_train,y_train) # Ваш код здесь 

Lasso(alpha=0.01, random_state=17)

**Посмотрите на коэффициенты модели и отранжируйте признаки по влиянию на качество вина. Какой признак "отвалился" первым, то есть наименее важен для объяснения целевого признака в модели Lasso?**

In [10]:
# lasso1_coef = pd.DataFrame # Ваш код здесь
# lasso1_coef.sort_values # Ваш код здесь
# lasso1.coef_,lasso1.intercept_
pd.DataFrame(lasso1.coef_, X_holdout.columns, columns=[
             "coef"]).sort_values(by="coef", ascending=False)


,coef
alcohol,0.352083
residual sugar,0.022231
free sulfur dioxide,0.005223
citric acid,0.000000
chlorides,-0.000000
density,-0.000000
pH,0.000000
sulphates,0.000000
total sulfur dioxide,-0.001108
fixed acidity,-0.059770


**Теперь определите лучшее значение $\alpha$ в процессе кросс-валидации 5-кратной кросс-валидации. Используйте LassoCV и random_state=17.**

In [10]:
alphas = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(random_state=17, cv=5, alphas=alphas)  # Ваш код здесь
lasso_cv.fit(X_holdout, y_holdout)  # Ваш код здесь


LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

In [11]:
lasso_cv.alpha_

3.696912707195032e-05

**Выведите коэффициенты "лучшего" Lasso в порядке убывания влияния на качество вина. **<br>
**<font color='red'>Вопрос 3:</font> Какой признак "обнулился первым" в настроенной модели LASSO?**

In [12]:
# lasso_cv_coef = pd.DataFrame # Ваш код здесь
# lasso_cv_coef.sort_values # Ваш код здесь
pd.DataFrame(lasso_cv.coef_, X_holdout.columns, columns=["coef"]).sort_values(
    by="coef", ascending=False)


,coef
sulphates,0.757690
alcohol,0.349845
citric acid,0.049090
residual sugar,0.022394
free sulfur dioxide,0.006857
density,-0.000000
total sulfur dioxide,-0.001323
pH,-0.003305
fixed acidity,-0.030808
volatile acidity,-1.908224


**Оцените среднеквадратичную ошибку модели на обучающей и тестовой выборках.**<br>
**<font color='red'>Вопрос 4:</font> Каковы среднеквадратичные ошибки настроенной LASSO-регрессии на обучающей и отложенной выборках?**

In [43]:
print("Mean squared error (train): %.3f"%(np.sqrt(mean_squared_error(y_train, lasso_cv.predict(X_train_scaled))))) # Ваш код здесь
print("Mean squared error (test): %.3f"%((mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))))  # Ваш код здесь


Mean squared error (train): 4.516
Mean squared error (test): 16.976


## Случайный лес

**Обучите случайный лес с параметрами "из коробки", фиксируя только random_state=17.**

In [14]:
forest = RandomForestRegressor(random_state=17)  # Ваш код здесь
forest.fit(X_train, y_train)  # Ваш код здесь


RandomForestRegressor(random_state=17)

**<font color='red'>Вопрос 5:</font> Каковы среднеквадратичные ошибки случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error' и остальными параметрами по умолчанию) и на отложенной выборке?**

In [44]:
print("Mean squared error (train): %.3f" %((mean_squared_error(y_train, forest.predict(X_train_scaled))))) # Ваш код здесь
print("Mean squared error (cv): %.3f" % (-cross_val_score(forest, X_train_scaled, y_train, scoring='neg_mean_squared_error',n_jobs=-1).mean()))  # Ваш код здесь
print("Mean squared error (test): %.3f" %((mean_squared_error(y_holdout, forest.predict(X_holdout_scaled))))) # Ваш код здесь


Mean squared error (train): 2.189
Mean squared error (cv): 0.414
Mean squared error (test): 2.184


**Настройте параметры min_samples_leaf и max_depth с помощью GridSearchCV и опять проверьте качество модели на кросс-валидации и на отложенной выборке.**

In [ ]:
forest_params = {'max_depth': list(range(12, 20)), 
                 'min_samples_leaf': list(range(1, 8)),
                 'max_features': list(range(6,12))}

locally_best_forest = GridSearchCV(forest,param_grid=forest_params,n_jobs=-1) # Ваш код здесь
locally_best_forest.fit(X_train,y_train) # Ваш код здесь

In [ ]:
locally_best_forest.best_params_, locally_best_forest.best_score_

**К сожалению, результаты  GridSearchCV не полностью воспроизводимы (могут отличаться на разных платформах даже при фиксировании *random_state*). Поэтому обучите лес с параметрами max_depth=19, max_features=7, и min_samples_leaf=1 (лучшие в моем случае).**<br>
**<font color='red'>Вопрос 6:</font> Каковы среднеквадратичные ошибки настроенного случайного леса на обучающей выборке, на кросс-валидации (cross_val_score с scoring='neg_mean_squared_error') и на отложенной выборке?**

In [19]:
# forest_local=GridSearchCV(locally_best_forest)
# Ваш код здесь

In [ ]:
print("Mean squared error (cv): %.3f" % (-cross_val_score(
    locally_best_forest, scoring='neg_mean_squared_error',X=X_train_scaled,y=y_train, n_jobs=-1)))  # Ваш код здесь
print("Mean squared error (test): %.3f" % ((mean_squared_error(
    y_holdout, locally_best_forest.predict(X_holdout_scaled)))))  # Ваш код здесь


**Оцените важность признаков с помощью случайного леса.**<br>
**<font color='red'>Вопрос 7:</font> Какой признак оказался главным в настроенной модели случайного леса?**

In [ ]:
rf_importance = pd.DataFrame # Ваш код здесь
rf_importance.sort_values # Ваш код здесь

**Сделайте выводы о качестве моделей и оценках влияния признаков на качество вина с помощью этих трех моделей.**